# Mean-Reversion Submission Raw Template

This simple template provides the shortest and quickest implementation of the strategy explained in the 10-minute Mean-Reversion Template.

In [ ]:
import xarray as xr
import qnt.data as qndata

data = qndata.load_data(min_date="2015-01-01",
                        dims=("time", "field", "asset"))

is_liquid = data.loc[::-1,"is_liquid",:]

prices = data.loc[::-1,"close",:]
avg_prices = prices.rolling({"time":100}).mean()

prices_shifted = prices.shift({"time":1})
returns = (abs((prices - prices_shifted)/prices_shifted).rolling({"time":10}).mean())

go_long = (prices < avg_prices * (1.0-returns)).astype(int)
go_short = (prices > avg_prices * (1.0+returns)).astype(int)

liq_avg_returns = (is_liquid > 0.0).astype(int)*(go_long-go_short)

weights = liq_avg_returns / abs(liq_avg_returns).sum("asset")
weights = weights.fillna(0.0)

qndata.write_output(weights)